In [1]:
import os
from pathlib import Path
import subprocess
import matplotlib.pyplot as plt
import pyemu
import flopy
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import geopandas as gpd
import numpy as np
from IPython.display import display
import time
import psutil

In [2]:
#set variables used below
cwd = os.getcwd()
pardir = os.pardir
base_model_dir = Path('../../Base_Model_for_Scenarios').resolve()
obs_locs_dir =  Path('../../Locations/Obs_locs').resolve()
truth_model_riv_obs_results = '../../Refined_Truth_Model/Modflow_Input_Files/riv_obs.csv'                         
print(cwd)
print(pardir)
print(base_model_dir)
print(obs_locs_dir)

C:\Users\farnut1\Desktop\How-Many-Realizations-main\Scenario_Notebook\run
..
C:\Users\farnut1\Desktop\How-Many-Realizations-main\Base_Model_for_Scenarios
C:\Users\farnut1\Desktop\How-Many-Realizations-main\Locations\Obs_locs


In [3]:
# Get number of physical cores, minus 2.
num_workers = psutil.cpu_count(logical=False)
print(num_workers)

20


### Inputs

In [4]:
# Loops through all scenarios
# 12 possible scenario groups
for Group in range(1,13):#13):
    print(Group)

    num_reals_list = [10, 25, 50, 100, 250, 500, 1000, 2000]
    #num_reals_list = [10, 50]  # For testing purposes
    
    obs_locsR25 =  obs_locs_dir / 'R25.csv'
    obs_locsR100 = obs_locs_dir / 'R100.csv'
    obs_locsC25 = obs_locs_dir / 'C25.csv'
    obs_locsC100 = obs_locs_dir / 'C100.csv'

    if Group == 1:
        gname = 'R25_pp10'
        obs_locs = obs_locsR25
        pp_space = 10
    if Group == 2:
        gname = 'R100_pp10'
        obs_locs = obs_locsR100
        pp_space = 10
    if Group == 3:
        gname = 'C25_pp10'
        obs_locs = obs_locsC25
        pp_space = 10
    if Group == 4:
        gname = 'C100_pp10'
        obs_locs = obs_locsC100
        pp_space = 10
    if Group == 5:
        gname = 'R25_pp25'
        obs_locs = obs_locsR25
        pp_space = 25
    if Group == 6:
        gname = 'R100_pp25'
        obs_locs = obs_locsR100
        pp_space = 25
    if Group == 7:
        gname = 'C25_pp25'
        obs_locs = obs_locsC25
        pp_space = 25
    if Group == 8:
        gname = 'C100_pp25'
        obs_locs = obs_locsC100
        pp_space = 25
    if Group == 9:    
        gname = 'R25_pp50'
        obs_locs = obs_locsR25
        pp_space = 50
    if Group == 10:
        gname = 'R100_pp50'
        obs_locs = obs_locsR100
        pp_space = 50
    if Group == 11:
        gname = 'C25_pp50'
        obs_locs = obs_locsC25
        pp_space = 50
    if Group == 12:
        gname = 'C100_pp50'
        obs_locs = obs_locsC100
        pp_space = 50

    for num_real_val in num_reals_list:
        num_reals=num_real_val
        print(num_reals)
    
        # specify a template directory (i.e. the PstFrom working folder)
        scenario_ws = os.path.join(pardir,f'{gname}'+"_real"+ f'{num_reals}')
        os.makedirs(scenario_ws, exist_ok=True)
        template_ws = os.path.join(cwd,'template_d')
    
        # Read in obs and pp locs, turn to dataframe
        obs_locs_df = pd.read_csv(obs_locs)
        
        # convert to geodataframe
        obs_locs_df = gpd.GeoDataFrame(obs_locs_df, geometry=gpd.points_from_xy(obs_locs_df['x'], obs_locs_df['y']), crs = 'EPSG:26915')
        
        # Get model_root_name from listing file
        for file in os.listdir(base_model_dir):
            if file.endswith(".dis"):
                disfile = file
        model_name = os.path.splitext(disfile)[0]
        print(model_name)
    
        sim = flopy.mf6.MFSimulation.load(f'{model_name}', sim_ws=base_model_dir, exe_name='mf6.exe')
        sim.set_sim_path(cwd)
        model = sim.get_model(f'{model_name}')
    
        # Intersect observations with the grid to get cellid
        ix = flopy.utils.gridintersect.GridIntersect(model.modelgrid, method="vertex", rtree=True)
        
        obs_cells = []
        for g in obs_locs_df.geometry:
            v = ix.intersect(g)
            obs_cells.append((0, v['cellids'][0][0].item(), v['cellids'][0][1].item()))
        
        # Add cell id to dataframe
        obs_locs_df['cellid'] = obs_cells
        
        # Add obsname column
        # this should already have been added by Export_Heads_Capture_flows.ipynb
        #obs_locs_df = obs_locs_df.reset_index()
        #obs_locs_df = obs_locs_df.rename(columns={'index':'obsname'})
        #obs_locs_df['obsname'] +=1
        #obs_locs_df['obsname'] = 'o' + obs_locs_df['obsname'].astype(str)
        obs_locs_df['obsname'] = obs_locs_df['site_name']
        
        # Add head column
        obs_locs_df['obstype'] = 'head'
        
        # Make new df for obs package
        obs_package_df = obs_locs_df[['obsname','obstype','cellid']]
        
        # Turn to recarray
        obs_package_data = obs_package_df.to_records(index=False)
        
        obs_recarray = {'head_obs.csv': obs_package_data}
    
        # Add head obs to model
        gwfobs = flopy.mf6.ModflowUtlobs(model, continuous=obs_recarray, filename=f'{model_name}.obs', pname='gwfobs_0')
    
        # Add riv flux obs to model
        # first add a boundname to the stress period data
        riv_spd = {0: model.riv.stress_period_data.data[0]}
        #temporarily recast to dataframe because recarrays are too hard to work with
        riv_spd[0] = pd.DataFrame(riv_spd[0])
        riv_spd[0]['boundname'] = 'river'
        # update the conductance to be more like the truth model as well
        riv_spd[0]['cond'] = 250
        riv_spd[0] = riv_spd[0].to_records(index=False)
        # input to the MODFLOW for RIV observations
        riv_obs = pd.DataFrame({
            'obsname': ['riv-flux'],
            'type': ['riv'],
            'id': ['river']  # the boundname that we made above; includes all RIV cells
        }).to_records(index=False)
        riv = flopy.mf6.modflow.mfgwfriv.ModflowGwfriv(
            model, save_flows=True, observations={'riv_obs.csv': riv_obs}, stress_period_data=riv_spd, boundnames=True,
            filename=f'{model.name}.riv', pname='riv_0')
        
        # edit GHB package conductance to be more like truth model
        ghb_spd = {0: model.ghb.stress_period_data.data[0]}
        #temporarily recast to dataframe because recarrays are too hard to work with
        ghb_spd[0] = pd.DataFrame(ghb_spd[0])
        # set GHB leakance at arbitrary large value of 1,000
        # (cond = leakance * area)
        ghb_spd[0]['cond'] = 1000 * 25**2  # assumes base model discretization of 25 units
        ghb_spd[0] = ghb_spd[0].to_records(index=False)
        ghb = flopy.mf6.modflow.mfgwfghb.ModflowGwfghb(
            model, save_flows=True, stress_period_data=ghb_spd,
            filename=f'{model.name}.ghb', pname='ghb_0')
        
        sim.simulation_data.max_columns_auto_set=False
        sim.simulation_data.max_columns_of_data=1
        sim.set_all_data_external()
        sim.write_simulation()
        sim.run_simulation()    
    
        # Pest Section
        # Inputs
        # external file with horizontal hydraulic conductivity values
        kh_array_file = f'{model_name}.npf_k.txt'
        # external file with recharge
        rcha_external_array_file = f'{model_name}.rcha_recharge_1.txt'
        # external file with riv conductance
        riv_external_file =f'{model_name}.riv_stress_period_data_1.txt'
        # external file with well flow rates
        wel_external_file = f'{model_name}.wel_stress_period_data_1.txt'
        
        # pestpp-ies settings
        ies_num_reals = num_reals
        pest_casename = model_name
        
        # Outputs
        template_ws = Path(f'{template_ws}')  # new model workspace folder created by pyEMU for the PEST run
        out_pst_file = template_ws / f"{pest_casename}.pst"
        
        model_grid_cell_spacing = model.modelgrid.delr[0].item()  # assuming a uniform grid    
    
        # Set up the PstFrom object
        pf = pyemu.utils.PstFrom(original_d=cwd, new_d=template_ws, remove_existing=True, longnames=True, spatial_reference=model.modelgrid,
                                 zero_based=False, tpl_subfolder='tpl')    
        
        # fix wrapped external array files
        for f in kh_array_file, rcha_external_array_file:
            wrapped_array = np.loadtxt(template_ws / f)
            reshaped_array = np.reshape(wrapped_array, model.modelgrid.shape[1:])
            np.savetxt(template_ws / f, reshaped_array)    
    
        # Add Head observations
        heads_df = pd.read_csv(template_ws / "head_obs.csv", index_col=0)
        pf.add_observations('head_obs.csv', insfile='head_obs.csv.ins', index_cols='time', use_cols=(list(heads_df.columns.values)), prefix='hds')
        # Add RIV observation
        riv_flux_df = pd.read_csv(template_ws / "riv_obs.csv", index_col=0)
        pf.add_observations('riv_obs.csv', insfile='riv_obs.csv.ins', index_cols='time', use_cols=['RIV-FLUX'], prefix='riv')
    
        # directly estimate a mean value for hydraulic conductivity across the model grid
        prop = 'kh'
        par_name_base = f"{prop}-mult-constant"
        pf.add_parameters(filenames=kh_array_file, par_type="constant",
                          par_name_base=par_name_base,
                          pargp=par_name_base, zone_array=None,
                          upper_bound=10,
                          lower_bound=0.1,
                          )    
    
        # set up horizontal hydraulic conductivity pilot points
        # Initial multiplier to apply to pilot points
        pp_parval1 = 1
        # set the upper and lower pilot point bounds
        # based on a desired number of log cycles
        # PEST++ or pyEMU will use the bounds to sample a log-normal distribution;
        # setting the bounds symetically ensures that the distribution will centered on parval1
        log_range = 3  # number of orders of magnitude between upper and lower pilot point bounds
        pp_lower_bound = 10**(np.log10(pp_parval1) - log_range/2)
        pp_upper_bound = 10**(np.log10(pp_parval1) + log_range/2)
        
        # geostructure for pilot points
        alpha = model_grid_cell_spacing * pp_space * 3
        v = pyemu.geostats.ExpVario(contribution=1.0, a=alpha)
        pp_gs = pyemu.geostats.GeoStruct(variograms=v, nugget=0, transform='log')
    
        # add pilot point parameters
        par_style = 'mult'  # only multipliers are supported for pilot points in pyEMU
        par_type = 'pilotpoints'
        par_name_base = f"{prop}-{par_style}-{par_type}"
        pf.add_parameters(
            filenames=f'{model_name}.npf_k.txt', 
            par_type=par_type, par_style=par_style,
            geostruct=pp_gs, pp_space=pp_space,    
            # all pilot points are in the same zone (with same geo-structure)
            zone_array=None, use_pp_zones=False,
            comment_char='#',  # comment indicator in model files
            par_name_base=par_name_base,  # basename for parameters that are set up
            pargp=par_name_base,  # Parameter group to assign pars to.
            # This is PESTs pargp but is also used to gather correlated parameters set up using multiple
            # add_parameters() calls (e.g. temporal pars) with common geostructs.
            lower_bound=pp_lower_bound, upper_bound=pp_upper_bound, 
            #ult_ubound=upper_bound,ult_lbound=lower_bound,
            )
    
        # Add a recharge multiplier
        prop = 'rech'
        par_name_base = f"{prop}-mult-constant"
        pf.add_parameters(filenames=rcha_external_array_file, par_type="constant",
                          par_name_base=par_name_base,
                          pargp=par_name_base, zone_array=None,
                          upper_bound=2., lower_bound=0.5
                          )
    
        # Parametrize River Package conductances
        par_name_base="rivcond-mult-constant"
        pf.add_parameters(filenames=riv_external_file, par_type="constant",
            par_name_base=par_name_base, pargp=par_name_base,
            upper_bound=10, lower_bound=0.1,
            index_cols={'i': 1, 'j': 2}, 
            use_cols=[4], # parametrize rates in the 4th column
            par_style="multiplier",
            comment_char='#')
    
        # Parametrize Well Package pumping rates
        par_name_base="wellrate-mult-constant"
        pf.add_parameters(filenames=wel_external_file, par_type="grid",
            par_name_base=par_name_base, pargp=par_name_base,
            upper_bound=1.2, lower_bound=0.8,
            index_cols={'i': 1, 'j': 2}, 
            use_cols=[3], # parametrize rates in the 4th column
            par_style="multiplier",
            comment_char='#')
    
        # Set up the PEST control dataset
        pst = pf.build_pst(f'{pest_casename}.pst')
        pardata = pst.parameter_data
    
        # Add 'observed' head values from 'truth' model
        obs_locs_df['obsnme'] = [f"oname:hds_otype:lst_usecol:{site_no}_time:1" for site_no in obs_locs_df['obsname']]
        obs_locs_df.index = obs_locs_df['obsnme']
        pst.observation_data['obsval'] = obs_locs_df['head']
        # Add noise to observation data of 0.25 meters
        pst.observation_data["standard_deviation"] = 0.25
        # 'observed' RIV flux
        riv_flux_truth_df = pd.read_csv(truth_model_riv_obs_results)
        riv_flux_truth = riv_flux_truth_df['RIV-FLUX'].values[0]
        pst.observation_data.loc['oname:riv_otype:lst_usecol:riv-flux_time:1', 'obsval'] = riv_flux_truth
        pst.observation_data.loc['oname:riv_otype:lst_usecol:riv-flux_time:1', 'standard_deviation'] =\
            np.abs(0.05 * riv_flux_truth)  # +/- 5% noise
        pst.observation_data.loc['oname:riv_otype:lst_usecol:riv-flux_time:1', 'weight'] =\
            1/np.abs(0.10 * riv_flux_truth)  # implies 10% std error
    
        # build spatial covariance
        # if you have more than about 35K pars, the cov matrix becomes hard to handle
        if pf.pst.npar < 35000: 
            cov = pf.build_prior(fmt='coo', filename=template_ws / "prior_cov.jcb")
            pst.pestpp_options["parcov"] = "prior_cov.jcb"
        
            # build the prior ensemble
            pe = pf.draw(num_reals=ies_num_reals, use_specsim=True, scale_offset=False) # draw parameters from the prior distribution
            pe.enforce() # enforces parameter bounds
            pe.to_binary(template_ws / "prior_pe.jcb")  #writes the paramter ensemble to binary file
            assert pe.shape[1] == np.sum(pardata.partrans != 'fixed')  
            pst.pestpp_options["ies_parameter_ensemble"] = "prior_pe.jcb"
    
        # pst options
        pst.control_data.noptmax = 4
        pst.pestpp_options["ies_num_reals"] = ies_num_reals
        pst.pestpp_options["ies_no_noise"] = "false"
        pst.pestpp_options["ies_bad_phi_sigma"] = 2.5
        #pst.pestpp_options['ies_save_binary'] = 'true'
        #pst.pestpp_options['ies_csv_by_reals'] = 'false'
    
        # write out PEST control file
        pf.pst.write(out_pst_file, version=2)
        print(f'wrote {out_pst_file}')
    
        # Add mf6 to forward run
        pf.mod_sys_cmds.append('mf6')
        pf.write_forward_run()
    
        # Run IES
        pyemu.os_utils.start_workers(template_ws, 'pestpp-ies', f'{model_name}.pst',
                                     num_workers=num_workers, worker_root=cwd, master_dir=scenario_ws)

1
10
fp_mf6_model
loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package riv...
    loading package ghb...
    loading package wel...
    loading package rch...
    loading package npf...
    loading package oc...
  loading solution package fp_mf6_model...
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package fp_mf6_model...
  writing model fp_mf6_model...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package wel_0...
    writing package rcha_0...
    writing package npf...
    writing package oc...
    writing package gwfobs_0...
    writing package riv_0...
INFORMATION: maxbound in ('gwf6', 'riv', 'dimensions') changed to 400 based on size of stress_period_data
    writing package obs_0...
    writing package ghb_0...
INFORMATION: maxbound in ('gwf